In [1]:
import sys
sys.path.append('./')
sys.path.append('../')

from interventions import three_operands

from transformers import AutoTokenizer, AutoModelForCausalLM
from omegaconf import DictConfig, OmegaConf
from tqdm.notebook import tqdm
import torch
import random
import yaml
import numpy as np
import pickle
import seaborn as sns
import re
import copy

# from llm_transparency_tool.models.tlens_model import TransformerLensTransparentLlm
# from llm_transparency_tool.models.transparent_llm import ModelInfo
# from llm_transparency_tool.routes.graph import GraphBuilder, build_full_graph, build_paths_to_predictions
import networkx as nx

import matplotlib.pyplot as plt
import warnings

%matplotlib inline

/home/scratch_local/esx208-907207/icl_mech_interp/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/scratch_local/esx208-907207/icl_mech_interp/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
mkdir -p failed for path /scratch_local/eickhoff/esx208/.cache/matplotlib: [Errno 13] Permission denied: '/scratch_local/eickhoff'
Matplotlib created a temporary cache directory at /tmp/matplotlib-q_qr62id because there was an issue with the default path (/scratch_local/eickhoff/esx208/.cache/matplotlib); it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to b

In [2]:
seed = 0
random.seed(seed)
torch.manual_seed(seed)
torch.set_grad_enabled(False)

In [3]:
# model_name = 'EleutherAI/pythia-12b-deduped-v0'
# model_name_lens = 'pythia-12b-deduped-v0'
# model_name = 'EleutherAI/pythia-6.9b-deduped-v0'
# model_name_lens = 'pythia-6.9b-deduped-v0'
# model_name = 'facebook/opt-1.3b'
# model_name_lens = 'opt-1.3b'
# model_name = 'openai-community/gpt2'
# model_name_lens = 'gpt2'
# model_name = 'facebook/opt-2.7b'
# model_name_lens = 'opt-2.7b'
model_name = 'facebook/opt-6.7b'
model_name_lens = 'opt-6.7b'
# model_name = 'facebook/opt-13b'
# model_name_lens = 'opt-13b'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
conf = OmegaConf.load('./conf/config.yaml')
conf.model = model_name
# conf.max_n = 9
conf.max_n = 20
# conf.n_shots = 1
# intervention_list = three_operands.get_arithmetic_data_three_operands(tokenizer, conf)
# intervention_list = pickle.load(open('/mnt/qb/work/eickhoff/esx208/arithmetic-lm/data/intervention_1_shots_max_20_arabic_further_templates.pkl', 'rb'))
# intervention_list = pickle.load(open('/mnt/qb/work/eickhoff/esx208/arithmetic-lm/data/information_flow_routes_results_opt_2_7b/base_data_arabic_opt_2_7.pkl', 'rb'))
# intervention_list = torch.load('/mnt/qb/work/eickhoff/esx208/arithmetic-lm/data/information_flow_routes_results_opt_2_7b/base_data_arabic_opt_2_7.pkl')
intervention_list = torch.load('/mnt/qb/work/eickhoff/esx208/arithmetic-lm/data/patching_results_opt_6_7b_old/intervention_list.pkl')

/tmp/ipykernel_2759805/1598515283.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  intervention_list = torch.load('/mnt/qb/work/eickhoff/esx208/arithmetic-lm/data/patchi

In [5]:
def calc_accuracy(intervention_list, model):
    accuracy_base = []
    accuracy_alt = []

    accuracy_base_tok = []
    accuracy_alt_tok = []

    for item in tqdm(intervention_list):
        # input_id_base = item.base_string_tok.to(model.device)
        # input_id_base = torch.tensor([tokenizer.bos_token_id] + item.base_string_tok_list).unsqueeze(0).to(model.device)
        
        # For other models
        # input_id_base = torch.tensor(tokenizer.encode(' ' + item.few_shots + item.base_string)).unsqueeze(0).to(model.device)
        # input_id_alt = item.alt_string_tok.to(model.device)
        
        # For other models and for testing the results without padding
        input_id_base = torch.tensor(tokenizer.encode(' ' + item.few_shots.lstrip() + item.base_string)).unsqueeze(0).to(model.device)
        input_id_alt = torch.tensor(tokenizer.encode(' ' + item.base_string.lstrip())).unsqueeze(0).to(model.device)
        
        # For pythia and for testing the results without padding
        # input_id_base = torch.tensor([tokenizer.bos_token_id] + tokenizer.encode(' ' + item.few_shots + item.base_string)).unsqueeze(0).to(model.device)
        # input_id_alt = torch.tensor([tokenizer.bos_token_id] + tokenizer.encode(' ' + item.base_string)).unsqueeze(0).to(model.device)

        output_base = model.generate(input_id_base, max_new_tokens=1, do_sample=False)[0, -1].cpu().numpy()
        output_alt = model.generate(input_id_alt, max_new_tokens=1, do_sample=False)[0, -1].cpu().numpy()
        output_base_str = tokenizer.decode(output_base)
        output_alt_str = tokenizer.decode(output_alt)
        correct_output_tok = item.res_base_tok[0]
        correct_output_str = item.res_base_string

        # print(output_base, output_alt, correct_output_tok)
        # correct_output_str = int(item.res_base_string)
        print('With icd: ', output_base_str, 'Without icd: ', output_alt_str, 'Correct: ', correct_output_str)
        print(tokenizer.decode(input_id_base[0]))
        print(tokenizer.decode(input_id_alt[0]))
        print('#' * 10)

        try:
            accuracy_base.append(int(output_base_str) == int(correct_output_str))
        except:
            accuracy_base.append(0)
        
        try:
            accuracy_alt.append(int(output_alt_str) == int(correct_output_str))
        except:
            accuracy_alt.append(0)
        
        accuracy_base_tok.append(output_base == correct_output_tok)
        accuracy_alt_tok.append(output_alt == correct_output_tok)

    return accuracy_base, accuracy_alt, accuracy_base_tok, accuracy_alt_tok

accuracy_base, accuracy_alt, accuracy_base_tok, accuracy_alt_tok = calc_accuracy(intervention_list, model)

  0%|          | 0/210 [00:00<?, ?it/s]

With icd:   19 Without icd:   23 Correct:  14
</s> 4 + 4 + 6 = 14. 3 + 5 + 6 =
</s> 3 + 5 + 6 =
##########
With icd:   20 Without icd:   28 Correct:  17
</s> 3 + 2 + 8 = 13. 3 + 7 + 7 =
</s> 3 + 7 + 7 =
##########
With icd:   10 Without icd:   12 Correct:  12
</s> 4 + 2 + 8 = 14. 8 + 2 + 2 =
</s> 8 + 2 + 2 =
##########
With icd:   18 Without icd:   24 Correct:  14
</s> 10 + 2 + 3 = 15. 2 + 6 + 6 =
</s> 2 + 6 + 6 =
##########
With icd:   19 Without icd:   23 Correct:  19
</s> 5 + 4 + 2 = 11. 8 + 6 + 5 =
</s> 8 + 6 + 5 =
##########
With icd:   11 Without icd:   13 Correct:  11
</s> 9 + 2 + 2 = 13. 6 + 2 + 3 =
</s> 6 + 2 + 3 =
##########
With icd:   15 Without icd:   27 Correct:  17
</s> 4 + 6 + 8 = 18. 2 + 9 + 6 =
</s> 2 + 9 + 6 =
##########
With icd:   23 Without icd:   23 Correct:  18
</s> 3 + 6 + 2 = 11. 5 + 10 + 3 =
</s> 5 + 10 + 3 =
##########
With icd:   24 Without icd:   28 Correct:  18
</s> 3 + 3 + 7 = 13. 4 + 9 + 5 =
</s> 4 + 9 + 5 =
##########
With icd:   24 Without icd:   24 C

In [6]:
print('Accuracy with icd, numbers equality', np.mean(accuracy_base))
print('Accuracy without icd, numbers equality', np.mean(accuracy_alt))

print('Accuracy with icd, tokens equality', np.mean(accuracy_base_tok))
print('Accuracy without icd, tokens equality', np.mean(accuracy_alt_tok))

Accuracy with icd, numbers equality 0.319047619047619
Accuracy without icd, numbers equality 0.23333333333333334
Accuracy with icd, tokens equality 0.319047619047619
Accuracy without icd, tokens equality 0.23333333333333334


In [7]:
def break_down_accuracy(intervention_list, accuracy):
    number_pattern = r"(?:\d+|one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty)"

    templates = [
        re.compile(rf"The result of {number_pattern} \* {number_pattern} \* {number_pattern} = {number_pattern}\."),
        re.compile(rf"The result of {number_pattern} times {number_pattern} times {number_pattern} = {number_pattern}\."),
        re.compile(rf"{number_pattern} \* {number_pattern} \* {number_pattern} = {number_pattern}\."),
        re.compile(rf"The result of {number_pattern} \+ {number_pattern} \+ {number_pattern} = {number_pattern}\."),
        re.compile(rf"{number_pattern} \+ {number_pattern} \+ {number_pattern} = {number_pattern}\."),
        re.compile(rf"The result of \({number_pattern} - {number_pattern}\) \* {number_pattern} = {number_pattern}\."),
        re.compile(rf"The result of {number_pattern} minus {number_pattern} minus {number_pattern} is {number_pattern}\.")
    ]

    accuracies_per_template = [[], [], [], [], [], [], []]

    for i in range(len(accuracy)):
        for j, template in enumerate(templates):
            if template.match(intervention_list[i].few_shots):
                accuracies_per_template[j].append(accuracy_base[i])
    
    return [np.sum(item) if item else 0 for item in accuracies_per_template]

In [8]:
break_down_accuracy(intervention_list, accuracy_base)

[21, 15, 7, 8, 6, 0, 1]

In [9]:
np.array(break_down_accuracy(intervention_list, accuracy_base)) / 30

array([0.7       , 0.5       , 0.23333333, 0.26666667, 0.2       ,
       0.        , 0.03333333])

Let's calculate scores on the dataset with word representation

In [10]:
if 'facebook/opt' in model_name:
    SYMBOLS = ["alpha", "beta", "lambda", "delta", "omega", "mu", "nu", "pi", "chi", "number", "res", "x", "y", "z", "a", "b", "c"]
    WORD_NUMBERS = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine",
        "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen", "twenty"
    ]
elif 'EleutherAI/pythia' in model_name:
    SYMBOLS = ["alpha", "beta", "gamma", "delta", "mu", "nu", "chi", "theta", "sigma", "number", "res", "x", "y", "z", "a", "b", "c"]
    WORD_NUMBERS = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine",
        "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen", "twenty"
    ]
    
for item in SYMBOLS:
    print(tokenizer.tokenize(' ' + item))
for item in WORD_NUMBERS:
    print(tokenizer.tokenize(' ' + item))

['Ġalpha']
['Ġbeta']
['Ġlambda']
['Ġdelta']
['Ġomega']
['Ġmu']
['Ġnu']
['Ġpi']
['Ġchi']
['Ġnumber']
['Ġres']
['Ġx']
['Ġy']
['Ġz']
['Ġa']
['Ġb']
['Ġc']
['Ġzero']
['Ġone']
['Ġtwo']
['Ġthree']
['Ġfour']
['Ġfive']
['Ġsix']
['Ġseven']
['Ġeight']
['Ġnine']
['Ġten']
['Ġeleven']
['Ġtwelve']
['Ġthirteen']
['Ġfourteen']
['Ġfifteen']
['Ġsixteen']
['Ġseventeen']
['Ġeighteen']
['Ġnineteen']
['Ġtwenty']


In [11]:
intervention_list[0].__dict__

{'op3_pos': 12,
 'operator_word': None,
 'operands_alt': '3 5 6',
 'operands_base': '3 5 6',
 'operator_pos': None,
 'op2_pos': 10,
 'op1_pos': 8,
 'res_alt_tok': [501],
 'res_base_tok': [501],
 'res_string': None,
 'res_base_string': '14',
 'res_alt_string': '14',
 'device': 'cpu',
 'multitoken': False,
 'is_llama': False,
 'is_opt': True,
 'is_bloom': False,
 'is_mistral': False,
 'is_persimmon': False,
 'representation': 'arabic',
 'extended_templates': True,
 'template_id': '-',
 'n_vars': 2,
 'base_string': '3 + 5 + 6 =',
 'alt_string': '3 + 5 + 6 =',
 'few_shots': '4 + 4 + 6 = 14. ',
 'few_shots_t2': ' ',
 'equation': '({x}+{y}+{z})',
 'enc': GPT2TokenizerFast(name_or_path='facebook/opt-6.7b', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={

In [12]:
def replace_numbers_with(intervention_list, conf, operands=False, result=False, few_shot=False, prompt=False, replacement = 'symbols'):
    counterfactual_intervention_list = []
    for intervention in tqdm(intervention_list):
        # Create copy of intervention
        new_intervention = copy.deepcopy(intervention)
        
        def replace(text, operands=operands, result=result):
            # Get few shot example and tokenize it
            text_tokenized = tokenizer.encode(text, add_special_tokens=False)        
            
            # Get positions of numbers
            number_positions = []
            for pos, item in enumerate(text_tokenized):
                if tokenizer.decode(item).strip().isnumeric():
                    number_positions.append(pos)
            
            number_positions_selected = []
            if operands:
                number_positions_selected += number_positions[:-1]
            if result:
                number_positions_selected += number_positions[-1:]
            
            # For all position in number positions randomly choose symbol to replace and 
            for pos in number_positions_selected:
                if replacement == 'symbols':
                    result_str = ' ' + random.choice(SYMBOLS)
                    new_result_enc = tokenizer.encode(result_str, add_special_tokens=False)[0]
                    text_tokenized[pos] = new_result_enc
                elif replacement == 'words':
                    number = int(tokenizer.decode(text_tokenized[pos]).strip())
                    num_word = ' ' + WORD_NUMBERS[number]
                    num_word_enc = tokenizer.encode(num_word, add_special_tokens=False)[0]
                    text_tokenized[pos] = num_word_enc
                elif replacement == 'random':
                    if result and not operands:
                        result_int = int(tokenizer.decode(text_tokenized[pos]).strip())
                        new_result = random.randint(1, conf.max_n)
                        while new_result == result_int:
                            new_result = random.randint(1, conf.max_n)
                        new_result_enc = tokenizer.encode(' ' + str(new_result), add_special_tokens=False)[0]
                        text_tokenized[pos] = new_result_enc
                    else:
                        raise NotImplementedError('Only supported for altering the result')
                else:
                    raise NotImplementedError(replacement + ' ' + 'not implemented!')
            
            return text_tokenized, tokenizer.decode(text_tokenized)
                
        
        if few_shot:
            few_shot_tokenized, few_shot_text = replace(intervention.few_shots, operands=operands, result=result)
            new_intervention.few_shots = few_shot_text
    
        if prompt:
            prompt_tokenized, prompt_text = replace(intervention.base_string, operands=operands, result=True)
            new_intervention.base_string_tok = prompt_tokenized
            new_intervention.base_string = prompt_text
            if result:
                result_tokenized, result_text = replace(intervention.res_base_string, operands=False, result=True)
                new_intervention.res_base_tok = result_tokenized
                new_intervention.res_base_string = result_text
                
        counterfactual_intervention_list.append(new_intervention)
    return counterfactual_intervention_list

In [13]:
intervention_list_texts = replace_numbers_with(intervention_list, conf, operands=True, result=True, few_shot=True, prompt=True, replacement='words')

  0%|          | 0/210 [00:00<?, ?it/s]

In [14]:
accuracy_base_words, accuracy_alt_words, accuracy_base_tok_words, accuracy_alt_tok_words = calc_accuracy(intervention_list_texts, model)

  0%|          | 0/210 [00:00<?, ?it/s]

With icd:   eleven Without icd:   nine Correct:   fourteen
</s> four + four + six = fourteen.  three + five + six =
</s> three + five + six =
##########
With icd:   twenty Without icd:   21 Correct:   seventeen
</s> three + two + eight = thirteen.  three + seven + seven =
</s> three + seven + seven =
##########
With icd:   fourteen Without icd:   sixteen Correct:   twelve
</s> four + two + eight = fourteen.  eight + two + two =
</s> eight + two + two =
##########
With icd:   twelve Without icd:   twelve Correct:   fourteen
</s> ten + two + three = fifteen.  two + six + six =
</s> two + six + six =
##########
With icd:   thirteen Without icd:   twenty Correct:   nineteen
</s> five + four + two = eleven.  eight + six + five =
</s> eight + six + five =
##########
With icd:   nine Without icd:   nine Correct:   eleven
</s> nine + two + two = thirteen.  six + two + three =
</s> six + two + three =
##########
With icd:   fifteen Without icd:   twenty Correct:   seventeen
</s> four + six + ei

In [18]:
print('Accuracy with icd, numbers equality', np.mean(accuracy_base_words))
print('Accuracy without icd, numbers equality', np.mean(accuracy_alt_words))

print('Accuracy with icd, tokens equality', np.mean(accuracy_base_tok_words))
print('Accuracy without icd, tokens equality', np.mean(accuracy_alt_tok_words))

Accuracy with icd, numbers equality 0.0
Accuracy without icd, numbers equality 0.0
Accuracy with icd, tokens equality 0.19523809523809524
Accuracy without icd, tokens equality 0.01904761904761905


In [19]:
break_down_accuracy(intervention_list_texts, accuracy_base_tok_words)

[21, 15, 0, 8, 0, 0, 1]

In [ ]:
(np.array(break_down_accuracy(intervention_list, accuracy_base)) + np.array(break_down_accuracy(intervention_list_texts, accuracy_base_tok_words))) / 60

array([0.7       , 0.5       , 0.11666667, 0.26666667, 0.1       ,
       0.        , 0.03333333])

: 